In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import sys
from utils.sequence import uniprot2sequence, encode_sequences
from utils.chem import *
from utils.parallel import *
from utils.sequence import encode_sequences
from utils.chem import get_mols, get_fp  
import pickle
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
from rdkit import Chem
from xgboost import XGBClassifier
from base_model import BaseModel
from preprocessor import Preprocessor
from barlow_twins import BarlowTwins
import numpy as np, pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from xgboost import XGBClassifier
import joblib
from SSCNN_dataset import NewDataset
from SSCNN_model import SSCNN_DTI
from torch.utils.data import DataLoader
import torch, pickle
from SSCNN_utils import get_one_bcm

In [ ]:

df_drugs = pd.read_csv("tb_final_filtrado.csv")

Creation of a dataset containing all drugs from the Papyrus dataset that interact with the following tuberculosis targets

### 50 most import targets

In [ ]:
target_50 = [
    "Enoyl-[acyl-carrier-protein] reductase",
    "Arabinosyltransferase A (embA)",
    "DNA-directed RNA polymerase beta chain",
    "Thymidylate kinase (TMK)",
    "3-oxoacyl-[acyl-carrier-protein] synthase III( mtFabH)",
    "3-oxoacyl-[acyl-carrier-protein] synthase 2",
    "3-oxoacyl-[acyl-carrier-protein] synthase 1",
    "Cell division protein FtsZ",
    "UDP-galactopiranose mutase (UGM)",
    "DNA girase (GyrA)",
    "1-deoxy-D-xylulose-5-phosphate synthase",
    "1-deoxy-D-xylulose 5-phosphate reductoisomerase",
    "Peptide deformylase",
    "Pantothenate synthetase",
    "Probable serine/threonine-protein kinase pknG",
    "UDP-galactofuranosyl transferase GlfT2",
    "Purine nucleoside phosphorylase",
    "Isocitrate lyase",
    "Cytochrome P450 121",
    "HTH-type transcriptional regulator EthR",
    "Thymidylate synthase thyX",
    "Serine/threonine-protein kinase pknB",
    "DNA ligase",
    "Ribonucleoside-diphosphate reductase subunit alpha",
    "ATP synthase",
    "Thioredoxin reductase",
    "2-C-methyl-D-erythritol 2 4-cyclodiphosphate synthase",
    "DNA gyrase subunit A/DNA gyrase subunit B",
    "Biotin--acetyl-CoA-carboxylase ligase",
    "FAD-dependent decaprenylphosphoryl-beta-D-ribofuranose 2-oxidase",
    "N-acetyltransferase Eis",
    "Dihydrolipoyllysine-residue acetyltransferase component of pyruvate dehydrogenase complex",
    "Leucine--tRNA ligase",
    "Polyketide synthase Pks13",
    "Serine/threonine-protein kinase PknA",
    "Inosine-5'-monophosphate dehydrogenase",
    "Isocitrate lyase 1",
    "Long-chain-fatty-acid--AMP ligase FadD32",
    "ATP-dependent Clp protease ATP-binding subunit ClpC1",
    "Tryptophan--tRNA ligase",
    "Coenzyme A biosynthesis bifunctional protein CoaBC",
    "ClpP1P2",
    "Type II NADH:quinone oxidoreductase Ndh",
    "Type II NADH:quinone oxidoreductase NdhA",
    "4'-phosphopantetheinyl transferase PptT",
    "Protein RecA",
    "Pyrazinamidase/nicotinamidase PncA",
    "DNA-directed RNA polymerase subunit beta'",
    "Catalase-peroxidase",
    "ATP synthase subunit c"
]

### 10 most importante targets:

In [ ]:
target_10 = ["Pantothenate synthetase",  
    "Arabinosyltransferase A",                 
    "DNA-directed RNA polymerase beta chain",         
    "Thymidylate kinase",
    "3-oxoacyl-[acyl-carrier-protein] synthase III",
    "3-oxoacyl-[acyl-carrier-protein] synthase 2",
    "3-oxoacyl-[acyl-carrier-protein] synthase 1",
    "Cell division protein FtsZ",
    "1-deoxy-D-xylulose-5-phosphate synthase",
    "Peptide deformylase"  
   
]

### Only isoniazid

In [ ]:
target_iso = ["Enoyl-[acyl-carrier-protein] reductase",  # alvo da isoniazida
]

### Dataset creation

In [ ]:

df_filtrado = df_drugs[df_drugs["Target Name"].isin(target_10)]

# df_filtrado.to_csv("Targets_10.csv", index=False)

### Merge drugs and targets

In [ ]:
df_drugs = pd.read_csv("Papyrus_Barlow.csv")  # ajuste o caminho
drugs = df_drugs["smiles"].astype(str).str.strip().dropna().drop_duplicates()

df_targets = pd.read_csv("Targets_10.csv")  # ajuste o caminho
targets = df_targets["Target Sequence"].astype(str).str.strip().dropna().drop_duplicates()


pairs = pd.merge(drugs.to_frame(name="smiles").assign(key=1),
                   targets.to_frame(name="Target Sequence").assign(key=1),
                   on="key").drop(columns="key")

pairs.to_csv("drug_target_pairs_results_10.csv", index=False)

### Barlow Twins run (We used the model previously trained with the papyrus+ dataset because it achieved the best results. )

In [ ]:
#in_csv = "drug_target_pairs_results_10.csv"
bt_dir = "/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model/stash/Papyrus_tb"           
xgb_json = "Papyrus_tb_barlowdti_xxl_model_tb_pap.json"
svm_pkl  = "Papyrus_tb_barlowdti_xxl_model_svm_tb_pap.pkl"
scaler_pkl = "Papyrus_tb_barlowdti_xxl_svm_scaler_tb_pap.pkl"

In [ ]:
preprocessor = Preprocessor(
    path="drug_target_pairs_results_10.csv",  
    radius=2,
    n_bits=1024,
    aa_embedding="prot_t5",
    num_workers=4
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BarlowTwins()
model.load_model("Papyrus_tb")  
model.move_to_device(device)

fps = preprocessor.fp
aas = preprocessor.aa

bt = BarlowTwins()
bt.load_model(bt_dir)                 
X = bt.zero_shot(fps, aas)           

# XGBoost 
xgb = XGBClassifier()
xgb.load_model(xgb_json)
probs_xgb = xgb.predict_proba(X)[:,1] 

# SVM 
scaler = joblib.load(scaler_pkl)
svm = joblib.load(svm_pkl)
Xs = scaler.transform(X)
probs_svm = svm.decision_function(Xs)
probs_svm_n = (probs_svm - probs_svm.min())/(probs_svm.max()-probs_svm.min()+1e-8)

probs = probs_xgb.flatten()

df_resultados = pd.DataFrame({
    "smiles": df["smiles"],
    "sequence": df["sequence"],
    "probabilidade": probs
})

df_resultados.to_csv("results_xgboost_10.csv", index=False)

probs_n = probs_svm_n.flatten()

df_resultados = pd.DataFrame({
    "smiles": df["smiles"],
    "sequence": df["sequence"],
    "probabilidade": probs_n
})

df_resultados.to_csv("resulsts_svm_10.csv", index=False)

### BCM-DTI run (We used the model previously trained with the papyrus+ dataset because it achieved the best results. )

In [ ]:
df = pd.read_csv("drug_target_pairs_results_10.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


with open("vocab/words2idx_d_Papyrus_TB.pkl","rb") as f: words2idx_d = pickle.load(f)
with open("words2idx_p_Papyrus_TB.pkl","rb") as f: words2idx_p = pickle.load(f)
max_drug_seq = 44
max_prot_seq = 462

smiles_list = df["smiles"].tolist()
prot_list   = df["sequence"].tolist()
labels_dummy = [0]*len(df) 
sm_frag, pr_frag, _ = get_one_bcm(smiles_list, prot_list, labels_dummy, decompose2="category", k=3)

with open("config_Papyrus_TB.json") as f:
    args_config = json.load(f)

dataset_inf = NewDataset(sm_frag, pr_frag, labels_dummy, words2idx_d, words2idx_p, max_drug_seq, max_prot_seq)
loader_inf  = DataLoader(dataset_inf, batch_size=64, shuffle=False)


model = SSCNN_DTI(args_config)  
model.load_state_dict(torch.load("Papyrus_TB.pkl", map_location=device))
model.to(device).eval()

scores = []
with torch.no_grad():
    for d, _, p, _ in loader_inf:
        s = model(d.long().to(device), p.long().to(device))
        scores.extend(s.squeeze().cpu().numpy().tolist())

df_out = df.copy()
df_out["score_bcmdti"] = scores
df_out.to_csv("Results_BCM_10.csv", index=False)



# Results

filter only those that were above 95 and 99% (to change the %, just change the filter, to change the number of targets just change the dataset)

In [ ]:

df_xg = pd.read_csv("results_xgboost_10.csv")
df_svm = pd.read_csv("results_vm_10.csv")
df_bcm = pd.read_csv("Results_BCM_10.csv.csv")

In [4]:
df_xg = df_xg[df_xg["probabilidade"] > 0.99]

The SVM results were removed because the results were very poor, which was to be expected after analysing the trained models.

In [ ]:
#df_svm = df_svm[df_svm["probabilidade_norm"] > 0.99]

In [15]:
df_bcm = df_bcm[df_bcm["score_bcmdti"] > 0.99]


### Merge

In [ ]:
cols = ["smiles", "sequence"]

final = df_xg.merge(df_bcm[cols], on=cols, how="inner")

print(duplicadas)



In [ ]:
final.to_csv("result_10_99.csv", index=False)